In [4]:
import geopy
from geopy.distance import distance
import pandas as pd
from rich.progress import track

In [5]:
pharmacies = pd.read_csv('../data/geocoded_pharmacies.csv')
records = pd.read_csv('../data/version3.csv')

/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (20,25) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [9]:
# instead of filtering on this field, we should filter on 
# a field from Pronob's dataset
filt_records = records[records['Opioid Related'] == True]
print(filt_records.shape)
filt_records.sample(1)

(7748, 33)


,Case Number,Date of Incident,Date of Death,Age,Gender,Race,Latino,Manner of Death,Primary Cause,Primary Cause Line A,...,location,Residence City,Residence_Zip,OBJECTID,cleaned_address,full_address,coded_lat,coded_long,coded_score,distance
23926,ME2017-00517,01/30/2017 01:21:00 PM,01/30/2017 01:53:00 PM,56.0,Female,Black,False,ACCIDENT,"COMBINED DRUG (4-ANPP, ACRYL FENTANYL, COCAINE...",NaN,...,"(41.70578918080517, -87.60804091023283)",Chicago,60628,69198,622 E. 104Th Street,622 E. 104Th Street Chicago 60628,41.705789,-87.608041,100.0,0.000023


In [10]:
pharm_lats = pharmacies.coded_lat.values
pharm_longs = pharmacies.coded_long.values
pharm_points = [(x, y) for x, y in zip(pharm_lats, pharm_longs) if pd.notna(x) and pd.notna(y)]

record_lats = filt_records.coded_lat.values
record_longs = filt_records.coded_long.values
record_points = [(x, y) for x, y in zip(record_lats, record_longs) if pd.notna(x) and pd.notna(y)]

In [11]:
distances = []
for p1 in track(record_points, total=len(record_points)):
    record_distances = []
    for p2 in pharm_points:
        d = distance(p1, p2).km
        record_distances.append(d)
    distances.append(min(record_distances))

In [12]:
sum(distances) / len(distances)

5.52858288455065